# 파이토치(PyTorch)로 구현하는 Mikolov 2010 RNN 언어 모델 (RNNLM)

본 노트북은 **"Recurrent neural network based language model" (Mikolov et al., 2010)** 논문을 기반으로 단순 RNN(Simple RNN)을 구현합니다. 

이 논문은 현대적인 딥러닝 기반 언어 모델링의 시초가 된 중요한 연구로, 가장 기본적인 형태의 RNN 구조를 제안했습니다.

---

## 📚 목차

1. [모델 구조 및 이론](#1-mikolov-rnnlm-모델-구조)
2. [환경 설정 및 라이브러리](#2-환경-설정)
3. [데이터 로드 및 전처리](#3-데이터-로드-및-전처리)
4. [RNN 모델 구현](#4-sigmoid-rnn-구현)
5. [학습](#5-학습-training)
6. [시각화 및 결과 분석](#6-결과-시각화)
7. [텍스트 생성](#7-텍스트-생성)

## 1. Mikolov RNNLM 모델 구조

### 1.1 모델 아키텍처

모델은 크게 **입력층(Input)**, **은닉층(Hidden)**, **출력층(Output)** 으로 구성됩니다.

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│  Input (w)  │ ──▶ │ Hidden (s)  │ ──▶ │ Output (y)  │
│  (One-hot)  │     │  (Sigmoid)  │     │  (Softmax)  │
└─────────────┘     └──────┬──────┘     └─────────────┘
                           │
                           ▼
                    ┌──────────────┐
                    │ Recurrent    │
                    │ Connection   │
                    └──────────────┘
```

### 1.2 핵심 수식

**은닉 상태 업데이트:**
$$ s(t) = \sigma(U \cdot w(t) + W \cdot s(t-1)) $$

**출력 계산:**
$$ y(t) = \text{softmax}(V \cdot s(t)) $$

### 1.3 변수 설명

| 기호 | 설명 | 차원 |
|:---:|:---|:---|
| $w(t)$ | 시간 $t$에서의 입력 단어 벡터 | $(V, 1)$ |
| $s(t)$ | 시간 $t$에서의 은닉 상태 | $(H, 1)$ |
| $y(t)$ | 시간 $t$에서의 출력 확률 분포 | $(V, 1)$ |
| $U$ | 입력 → 은닉 가중치 | $(H, V)$ |
| $W$ | 은닉 → 은닉 가중치 (순환 연결) | $(H, H)$ |
| $V$ | 은닉 → 출력 가중치 | $(V, H)$ |

### 1.4 왜 Sigmoid인가?

PyTorch의 기본 `nn.RNN`은 **Tanh** 활성화 함수를 사용하지만, **Mikolov의 원논문에서는 Sigmoid**를 사용했습니다.

- **Sigmoid**: $\sigma(x) = \frac{1}{1+e^{-x}}$, 출력 범위 $(0, 1)$
- **Tanh**: $\tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}$, 출력 범위 $(-1, 1)$

본 구현에서는 논문의 스펙을 정확히 따르기 위해 **Custom Sigmoid RNN Cell**을 직접 정의합니다.

## 2. 환경 설정

In [ ]:
import time
from collections import Counter
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset
import matplotlib.pyplot as plt

# 재현 가능성을 위한 시드 고정
SEED = 123
torch.manual_seed(SEED)
np.random.seed(SEED)

# Device 설정 (CUDA > MPS > CPU 우선순위)
def get_device() -> torch.device:
    """사용 가능한 최적의 디바이스 반환"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    elif torch.backends.mps.is_available():
        return torch.device('mps')
    return torch.device('cpu')

device = get_device()
print(f"사용 장치(Device): {device}")
print(f"PyTorch 버전: {torch.__version__}")

In [ ]:
class Config:
    """모델 및 학습 하이퍼파라미터 설정"""
    
    # 데이터 설정
    dataset_name: str = "Salesforce/wikitext"
    dataset_config: str = "wikitext-2-raw-v1"
    min_count: int = 5  # 최소 출현 빈도
    
    # 모델 하이퍼파라미터 (논문: Hidden Layer 30~500)
    embed_size: int = 100   # 임베딩 차원
    hidden_size: int = 100  # 은닉 상태 차원
    
    # 학습 설정
    batch_size: int = 32
    time_size: int = 35     # Truncated BPTT 길이
    max_epoch: int = 10
    learning_rate: float = 0.1  # SGD 학습률
    clip_grad: float = 0.25     # Gradient Clipping 임계값
    
    def __repr__(self):
        return (
            f"Config(\n"
            f"  embed_size={self.embed_size}, hidden_size={self.hidden_size},\n"
            f"  batch_size={self.batch_size}, time_size={self.time_size},\n"
            f"  max_epoch={self.max_epoch}, lr={self.learning_rate}\n"
            f")"
        )

config = Config()
print(config)

## 3. 데이터 로드 및 전처리

**Wikitext-2** 데이터셋을 사용합니다. 이 데이터셋은 위키피디아에서 추출된 양질의 텍스트로 구성되어 있으며, 언어 모델링 벤치마크로 널리 사용됩니다.

### 전처리 과정:
1. 데이터셋 다운로드 및 로드
2. 단어 빈도 카운팅
3. 최소 빈도 기준으로 어휘 사전 구축
4. 단어 → ID 변환

In [ ]:
print("데이터셋 다운로드 및 로드 중...")
ds = load_dataset(config.dataset_name, config.dataset_config)
print(f"데이터셋 로드 완료!")
print(f"   - Train: {len(ds['train'])} 문장")
print(f"   - Validation: {len(ds['validation'])} 문장")
print(f"   - Test: {len(ds['test'])} 문장")

In [ ]:
def build_vocab(
    dataset, 
    splits: List[str] = ['train', 'validation', 'test'],
    min_count: int = 5
) -> Tuple[Dict[str, int], Dict[int, str]]:
    """
    데이터셋에서 단어 빈도를 카운팅하고 어휘 사전을 구축합니다.
    
    Args:
        dataset: HuggingFace 데이터셋
        splits: 사용할 데이터셋 분할
        min_count: 최소 출현 빈도
        
    Returns:
        word_to_id: 단어 → ID 매핑
        id_to_word: ID → 단어 매핑
    """
    counter = Counter()
    
    for split in splits:
        for line in dataset[split]['text']:
            words = line.strip().lower().split()
            counter.update(words)
    
    # <unk> 토큰은 항상 0번 ID
    word_to_id = {'<unk>': 0}
    id_to_word = {0: '<unk>'}
    
    # 빈도순으로 정렬하여 ID 부여
    sorted_words = sorted(counter.items(), key=lambda x: x[1], reverse=True)
    valid_words = [word for word, count in sorted_words if count >= min_count]
    
    for word in valid_words:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word
    
    return word_to_id, id_to_word

# 어휘 사전 구축
word_to_id, id_to_word = build_vocab(ds, min_count=config.min_count)
vocab_size = len(word_to_id)

print(f"\n 어휘 사전 통계:")
print(f"   - 전체 어휘 크기: {vocab_size:,}")
print(f"   - 최소 출현 빈도: {config.min_count}")

In [ ]:
def convert_to_ids(
    dataset, 
    split: str, 
    word_to_id: Dict[str, int]
) -> np.ndarray:
    """
    텍스트 데이터를 단어 ID 시퀀스로 변환합니다.
    
    Args:
        dataset: HuggingFace 데이터셋
        split: 데이터 분할 ('train', 'validation', 'test')
        word_to_id: 단어 → ID 매핑
        
    Returns:
        ids: 단어 ID 배열
    """
    ids = []
    unk_id = word_to_id['<unk>']
    
    for line in dataset[split]['text']:
        words = line.strip().lower().split()
        if not words:
            continue
        ids.extend([word_to_id.get(w, unk_id) for w in words])
    
    return np.array(ids, dtype=np.int64)

# 각 분할에 대해 ID 변환
corpus_train = convert_to_ids(ds, 'train', word_to_id)
corpus_valid = convert_to_ids(ds, 'validation', word_to_id)
corpus_test = convert_to_ids(ds, 'test', word_to_id)

print(f"\n 코퍼스 크기:")
print(f"   - Train: {len(corpus_train):,} tokens")
print(f"   - Validation: {len(corpus_valid):,} tokens")
print(f"   - Test: {len(corpus_test):,} tokens")

## 4. Sigmoid RNN 구현

PyTorch의 `nn.RNN`은 기본적으로 `tanh` 혹은 `relu`만 지원합니다. Mikolov 논문의 스펙을 따르기 위해 **Sigmoid 활성화 함수**를 사용하는 RNN Layer를 직접 구현합니다.

### 구현 수식

$$h_t = \sigma(x_t W_{ih}^T + b_{ih} + h_{t-1} W_{hh}^T + b_{hh})$$

여기서:
- $x_t$: 시간 $t$의 입력 (임베딩 벡터)
- $h_{t-1}$: 이전 시간의 은닉 상태
- $W_{ih}$, $W_{hh}$: 가중치 행렬
- $b_{ih}$, $b_{hh}$: 편향

In [ ]:
class MikolovRNNLayer(nn.Module):
    """
    Mikolov 논문 스펙에 맞춘 Sigmoid RNN Layer.
    
    일반적인 RNN과 달리 Sigmoid 활성화 함수를 사용합니다.
    
    Args:
        input_size: 입력 차원
        hidden_size: 은닉 상태 차원
    """
    
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.hidden_size = hidden_size
        
        # 입력 → 은닉 변환 (U 행렬에 해당)
        self.i2h = nn.Linear(input_size, hidden_size)
        # 은닉 → 은닉 변환 (W 행렬에 해당, 순환 연결)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        # Sigmoid 활성화 함수
        self.activation = nn.Sigmoid()
        
    def forward(
        self, 
        x: torch.Tensor, 
        h_prev: Optional[torch.Tensor] = None
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        순전파 계산.
        
        Args:
            x: 입력 텐서 (batch, time_size, input_size)
            h_prev: 초기 은닉 상태 (batch, hidden_size), None이면 0으로 초기화
            
        Returns:
            outputs: 모든 시간 단계의 은닉 상태 (batch, time_size, hidden_size)
            h_last: 마지막 은닉 상태 (batch, hidden_size)
        """
        batch_size, time_size, _ = x.shape
        
        # 초기 은닉 상태가 없으면 0으로 초기화
        if h_prev is None:
            h_prev = torch.zeros(batch_size, self.hidden_size, device=x.device)
        
        h = h_prev
        outputs = []
        
        # 각 시간 단계별로 순차 계산
        for t in range(time_size):
            x_t = x[:, t, :]
            # h_t = sigmoid(U * x_t + W * h_{t-1})
            h = self.activation(self.i2h(x_t) + self.h2h(h))
            outputs.append(h.unsqueeze(1))
        
        # 전체 시퀀스 출력: (batch, time, hidden)
        outputs = torch.cat(outputs, dim=1)
        
        return outputs, h

In [ ]:
class MikolovRNNLM(nn.Module):
    """
    Mikolov 2010 RNN 언어 모델.
    
    구조: Embedding → Sigmoid RNN → Linear → Softmax
    
    Args:
        vocab_size: 어휘 크기
        embed_size: 임베딩 차원
        hidden_size: 은닉 상태 차원
    """
    
    def __init__(self, vocab_size: int, embed_size: int, hidden_size: int):
        super().__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        
        # 1. 입력층 (Embedding)
        # 논문의 One-hot 입력을 효율적으로 구현 (수학적으로 동등)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        # 2. 은닉층 (Sigmoid RNN)
        self.rnn = MikolovRNNLayer(embed_size, hidden_size)
        
        # 3. 출력층 (Linear → Softmax는 CrossEntropyLoss에서 처리)
        self.fc = nn.Linear(hidden_size, vocab_size)
        
        # 가중치 초기화
        self._init_weights()
        
    def _init_weights(self, init_range: float = 0.1):
        """가중치를 균등 분포로 초기화"""
        self.embedding.weight.data.uniform_(-init_range, init_range)
        self.fc.weight.data.uniform_(-init_range, init_range)
        self.fc.bias.data.zero_()
        
    def forward(
        self, 
        x: torch.Tensor, 
        h: Optional[torch.Tensor] = None
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        순전파.
        
        Args:
            x: 입력 단어 ID (batch, time_size)
            h: 초기 은닉 상태
            
        Returns:
            out: 로짓 (batch, time_size, vocab_size)
            h_new: 새로운 은닉 상태
        """
        # Embedding Lookup
        embeds = self.embedding(x)  # (batch, time, embed_size)
        
        # RNN 계산
        out, h_new = self.rnn(embeds, h)  # (batch, time, hidden_size)
        
        # 출력층
        out = self.fc(out)  # (batch, time, vocab_size)
        
        return out, h_new
    
    def get_num_params(self) -> int:
        """학습 가능한 파라미터 수 반환"""
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

In [ ]:
# 모델 생성 및 정보 출력
model = MikolovRNNLM(vocab_size, config.embed_size, config.hidden_size).to(device)

print("모델 구조:")
print(model)
print(f"\n 총 파라미터 수: {model.get_num_params():,}")

## 5. 학습 (Training)

### 5.1 Truncated BPTT (Backpropagation Through Time)

RNN의 역전파는 시간을 거슬러 올라가며 그래디언트를 계산합니다. 전체 시퀀스에 대해 BPTT를 수행하면 메모리와 계산 비용이 크므로, **Truncated BPTT**를 사용합니다.

```
전체 시퀀스:  [w1, w2, w3, w4, w5, w6, w7, w8, ...]
                    │         │         │
Truncated:   [chunk1]   [chunk2]   [chunk3]  ...
```

### 5.2 학습 설정

- **Optimizer**: SGD (논문 스펙)
- **Loss**: Cross-Entropy
- **Gradient Clipping**: 그래디언트 폭발 방지

In [ ]:
def get_batch(
    corpus: np.ndarray,
    batch_size: int,
    time_size: int,
    time_idx: int,
    offsets: List[int],
    data_size: int
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    학습용 미니배치를 생성합니다.
    
    Args:
        corpus: 전체 코퍼스 (단어 ID 배열)
        batch_size: 배치 크기
        time_size: 시퀀스 길이 (BPTT 길이)
        time_idx: 현재 시간 인덱스
        offsets: 각 배치 샘플의 시작 오프셋
        data_size: 코퍼스 크기
        
    Returns:
        inputs: 입력 시퀀스 (batch, time_size)
        targets: 타겟 시퀀스 (batch, time_size)
    """
    batch_x, batch_t = [], []
    
    for i in range(batch_size):
        ptr = (offsets[i] + time_idx) % data_size
        indices = [(ptr + t) % data_size for t in range(time_size + 1)]
        raw_data = corpus[indices]
        batch_x.append(raw_data[:-1])   # 입력: 0 ~ time_size-1
        batch_t.append(raw_data[1:])    # 타겟: 1 ~ time_size (다음 단어)
    
    return (
        torch.tensor(np.array(batch_x), dtype=torch.long),
        torch.tensor(np.array(batch_t), dtype=torch.long)
    )

In [ ]:
def evaluate(
    model: nn.Module,
    corpus: np.ndarray,
    batch_size: int,
    time_size: int,
    criterion: nn.Module
) -> float:
    """
    모델을 평가하고 평균 손실을 반환합니다.
    
    Args:
        model: RNN 모델
        corpus: 평가용 코퍼스
        batch_size: 배치 크기
        time_size: 시퀀스 길이
        criterion: 손실 함수
        
    Returns:
        avg_loss: 평균 손실
    """
    model.eval()
    data_size = len(corpus)
    max_iters = data_size // (batch_size * time_size)
    jump = (data_size - 1) // batch_size
    offsets = [i * jump for i in range(batch_size)]
    
    total_loss = 0.0
    hidden = None
    
    with torch.no_grad():
        for iter_idx in range(max_iters):
            inputs, targets = get_batch(
                corpus, batch_size, time_size, 
                iter_idx * time_size, offsets, data_size
            )
            inputs, targets = inputs.to(device), targets.to(device)
            
            outputs, hidden = model(inputs, hidden)
            loss = criterion(
                outputs.reshape(-1, model.vocab_size),
                targets.reshape(-1)
            )
            total_loss += loss.item()
            
            if hidden is not None:
                hidden = hidden.detach()
    
    return total_loss / max_iters if max_iters > 0 else float('inf')

In [ ]:
# 학습 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=config.learning_rate)

# 배치 샘플링을 위한 오프셋 계산
data_size = len(corpus_train)
max_iters = data_size // (config.batch_size * config.time_size)
jump = (data_size - 1) // config.batch_size
offsets = [i * jump for i in range(config.batch_size)]

# 학습 기록
history = {
    'train_loss': [],
    'train_ppl': [],
    'valid_loss': [],
    'valid_ppl': []
}

print(f"\n 학습 시작!")
print(f"   - 에폭 수: {config.max_epoch}")
print(f"   - 배치 크기: {config.batch_size}")
print(f"   - 이터레이션/에폭: {max_iters:,}")
print("=" * 70)

In [ ]:
start_time = time.time()

for epoch in range(config.max_epoch):
    model.train()
    hidden = None
    total_loss = 0.0
    
    for iter_idx in range(max_iters):
        # 미니배치 생성
        time_idx = iter_idx * config.time_size
        inputs, targets = get_batch(
            corpus_train, config.batch_size, config.time_size,
            time_idx, offsets, data_size
        )
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Truncated BPTT: 이전 은닉 상태 분리
        if hidden is not None:
            hidden = hidden.detach()
        
        # 순전파
        outputs, hidden = model(inputs, hidden)
        loss = criterion(
            outputs.reshape(-1, vocab_size),
            targets.reshape(-1)
        )
        
        # 역전파
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient Clipping (그래디언트 폭발 방지)
        torch.nn.utils.clip_grad_norm_(model.parameters(), config.clip_grad)
        
        # 파라미터 업데이트
        optimizer.step()
        
        total_loss += loss.item()
    
    # 에폭 통계
    train_loss = total_loss / max_iters
    train_ppl = np.exp(train_loss)
    
    # 검증 세트 평가
    valid_loss = evaluate(
        model, corpus_valid, config.batch_size, 
        config.time_size, criterion
    )
    valid_ppl = np.exp(valid_loss)
    
    # 기록 저장
    history['train_loss'].append(train_loss)
    history['train_ppl'].append(train_ppl)
    history['valid_loss'].append(valid_loss)
    history['valid_ppl'].append(valid_ppl)
    
    elapsed = time.time() - start_time
    print(
        f"| Epoch {epoch+1:2d}/{config.max_epoch} "
        f"| Train Loss {train_loss:.4f} | Train PPL {train_ppl:8.2f} "
        f"| Valid Loss {valid_loss:.4f} | Valid PPL {valid_ppl:8.2f} "
        f"| Time {elapsed:6.1f}s |"
    )

print("=" * 70)
print(f"학습 완료! 총 소요 시간: {time.time() - start_time:.1f}초")

In [ ]:
# 모델 저장
model_path = 'mikolov_rnn.pth'
torch.save({
    'model_state_dict': model.state_dict(),
    'config': {
        'vocab_size': vocab_size,
        'embed_size': config.embed_size,
        'hidden_size': config.hidden_size
    },
    'history': history
}, model_path)
print(f" 모델 저장 완료: {model_path}")

## 6. 결과 시각화

학습 과정에서의 손실(Loss)과 Perplexity(PPL) 변화를 시각화합니다.

### Perplexity (PPL)란?

Perplexity는 언어 모델의 성능을 측정하는 지표로, 모델이 다음 단어를 예측할 때의 "혼란도"를 나타냅니다.

$$PPL = e^{\text{Cross-Entropy Loss}}$$

- **낮을수록 좋음**: 모델이 다음 단어를 더 잘 예측
- **해석**: PPL=100이면, 모델이 각 위치에서 평균 100개의 단어 중에서 고르는 것처럼 "혼란"해 함

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss 그래프
ax1 = axes[0]
epochs = range(1, len(history['train_loss']) + 1)
ax1.plot(epochs, history['train_loss'], 'b-o', label='Train Loss', linewidth=2, markersize=6)
ax1.plot(epochs, history['valid_loss'], 'r-s', label='Valid Loss', linewidth=2, markersize=6)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Training & Validation Loss', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.set_xticks(epochs)

# PPL 그래프
ax2 = axes[1]
ax2.plot(epochs, history['train_ppl'], 'b-o', label='Train PPL', linewidth=2, markersize=6)
ax2.plot(epochs, history['valid_ppl'], 'r-s', label='Valid PPL', linewidth=2, markersize=6)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Perplexity', fontsize=12)
ax2.set_title('Training & Validation Perplexity', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)
ax2.set_xticks(epochs)

plt.tight_layout()
plt.savefig('training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n 최종 결과:")
print(f"   - Train PPL: {history['train_ppl'][-1]:.2f}")
print(f"   - Valid PPL: {history['valid_ppl'][-1]:.2f}")

## 7. 텍스트 생성

학습된 모델을 사용하여 텍스트를 생성합니다. **Temperature** 파라미터로 생성의 다양성을 조절할 수 있습니다.

- **Temperature = 1.0**: 원래 확률 분포 사용
- **Temperature < 1.0**: 더 확실한 (고빈도) 단어 선호
- **Temperature > 1.0**: 더 다양한 (저빈도 포함) 단어 허용

In [ ]:
def generate_text(
    model: nn.Module,
    start_word: str,
    word_to_id: Dict[str, int],
    id_to_word: Dict[int, str],
    length: int = 50,
    temperature: float = 1.0
) -> str:
    """
    학습된 모델로 텍스트를 생성합니다.
    
    Args:
        model: 학습된 RNN 모델
        start_word: 시작 단어
        word_to_id: 단어 → ID 매핑
        id_to_word: ID → 단어 매핑
        length: 생성할 단어 수
        temperature: 샘플링 온도 (높을수록 다양함)
        
    Returns:
        생성된 텍스트
    """
    model.eval()
    
    # 시작 단어 확인
    if start_word.lower() not in word_to_id:
        return f"'{start_word}'는 어휘에 없는 단어입니다."
    
    # 초기 상태 설정
    input_id = torch.tensor(
        [[word_to_id[start_word.lower()]]], 
        device=device
    )
    hidden = None
    result = [start_word.lower()]
    
    with torch.no_grad():
        for _ in range(length):
            # 순전파
            output, hidden = model(input_id, hidden)
            
            # Temperature 적용 및 확률 계산
            logits = output.squeeze() / temperature
            probs = torch.softmax(logits, dim=0)
            
            # 확률적 샘플링
            next_id = torch.multinomial(probs, 1).item()
            result.append(id_to_word[next_id])
            
            # 다음 입력 준비
            input_id = torch.tensor([[next_id]], device=device)
    
    return ' '.join(result)


# 텍스트 생성 예시
print("\n 생성된 텍스트:\n")
print("="*60)

for start_word in ['the', 'in', 'he', 'it']:
    if start_word in word_to_id:
        generated = generate_text(
            model, start_word, word_to_id, id_to_word,
            length=30, temperature=0.8
        )
        print(f"\n[{start_word}]로 시작:")
        print(f"  {generated}")

print("\n" + "="*60)

In [ ]:
# Temperature에 따른 생성 결과 비교
print("\n Temperature에 따른 생성 결과 비교:\n")

start = 'the'
for temp in [0.5, 0.8, 1.0, 1.2]:
    text = generate_text(
        model, start, word_to_id, id_to_word,
        length=20, temperature=temp
    )
    print(f"Temperature={temp}: {text}")
    print()